In [4]:

import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import os
import pyproj

# --- NEW: Fix PROJ Environment for Micromamba/Conda ---
try:
    # Try to find the database path automatically based on your Micromamba environment
    prefix = os.environ.get('CONDA_PREFIX')
    if not prefix:
        # Fallback to the path seen in your error message
        prefix = '/opt/homebrew/Cellar/micromamba/2.4.0/envs/lhd-environment'

    proj_path = os.path.join(prefix, 'share', 'proj')
    if os.path.exists(proj_path):
        os.environ['PROJ_LIB'] = proj_path
        pyproj.datadir.set_data_dir(proj_path)
except Exception:
    pass

# 1. Load the Excel Master List
excel_path = '/Users/kennyquintana/Downloads/merged_sites_clean.xlsx'
df_sites = pd.read_excel(excel_path, sheet_name='Sites')
df_sites.columns = [c.lower() for c in df_sites.columns]

nhd_to_tdx_list = []
tdx_to_nhd_list = []

print(f"Starting spatial mapping for {len(df_sites)} sites...")

for index, row in tqdm(df_sites.iterrows(), total=len(df_sites)):
    try:
        nhd_path = row.get('flowline_path_nhd')
        tdx_path = row.get('flowline_path_tdx')

        if pd.isna(nhd_path) or pd.isna(tdx_path):
            continue
        if not os.path.exists(str(nhd_path)) or not os.path.exists(str(tdx_path)):
            continue

        nhd = gpd.read_file(nhd_path)
        tdx = gpd.read_file(tdx_path)

        # Auto-lowercase headers
        nhd.columns = [c.lower() for c in nhd.columns]
        tdx.columns = [c.lower() for c in tdx.columns]

        # RE-INTRODUCED PROJECTION:
        # Using a meter-based projection for accurate 'nearest' calculations.
        # If 5070 still fails, it will fall back to the degrees (with the warning)
        try:
            nhd = nhd.to_crs(epsg=5070)
            tdx = tdx.to_crs(epsg=5070)
        except Exception:
            # Fallback: just ensure they match each other if the DB error persists
            if nhd.crs != tdx.crs:
                tdx = tdx.to_crs(nhd.crs)

        # 4. Spatial Join (Nearest TDX line for every NHD segment)
        mapped = gpd.sjoin_nearest(nhd, tdx, distance_col="dist")

        # Determine TDX ID column (usually 'linkno', 'river_id', or 'comid')
        possible_ids = ['linkno', 'river_id', 'comid']
        tdx_id_col = next((c for c in possible_ids if c in mapped.columns), mapped.columns[-1])

        # --- MAP 1: NHD -> GEOGLOWS (Many-to-One) ---
        map1 = mapped[['nhdplusid', tdx_id_col]].copy()
        nhd_to_tdx_list.append(map1)

        # --- MAP 2: GEOGLOWS -> NHD (One-to-One Outlet) ---
        # Use hydroseq if available, otherwise just pick one
        sort_col = 'hydroseq' if 'hydroseq' in mapped.columns else mapped.columns[0]

        map2 = mapped.sort_values(sort_col, ascending=True) \
                     .drop_duplicates(subset=[tdx_id_col])

        map2 = map2[[tdx_id_col, 'nhdplusid']].copy()
        tdx_to_nhd_list.append(map2)

    except Exception as e:
        print(f"\nError at index {index} (Site ID: {row.get('site_id')}): {e}")

# 5. Concatenate and Save
if nhd_to_tdx_list:
    final_nhd_to_tdx = pd.concat(nhd_to_tdx_list).drop_duplicates()
    final_tdx_to_nhd = pd.concat(tdx_to_nhd_list).drop_duplicates()

    final_nhd_to_tdx.to_csv('nhd_to_geoglows_crosswalk.csv', index=False)
    final_tdx_to_nhd.to_csv('geoglows_to_nhd_crosswalk.csv', index=False)
    print(f"\nSuccess! Mapped {len(final_nhd_to_tdx)} unique segments.")
else:
    print("\nNo data was successfully processed.")

Starting spatial mapping for 256 sites...


100%|██████████| 256/256 [00:36<00:00,  6.96it/s]


Success! Mapped 761 unique segments.
